In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

In [2]:
model2 = tf.keras.models.load_model('./saved_models/my_model.h5')
model1 = tf.keras.models.load_model('./saved_models/keras_model.h5')

In [3]:
model1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 1280)              410208    
                                                                 
 sequential_7 (Sequential)   (None, 2)                 128300    
                                                                 
Total params: 538,508
Trainable params: 524,428
Non-trainable params: 14,080
_________________________________________________________________


In [4]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 512)               32113152  
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 34,376,266
Trainable params: 32,118,282
Non-trainable params: 2,257,984
_________________________________________________________________


In [5]:
def inference_model(path):
  img = path
  img = image.load_img(img, target_size=(224, 224))  # Adjust target size as per your model's input shape
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.  # Normalize the image data
  
  predictions = model1.predict(img_array)
  predicted_class = np.argmax(predictions)
  classes = ['nails', 'not nails']
  
  if classes[predicted_class] == 'nails':
    predictions = model2.predict(img_array)
    predicted_class = np.argmax(predictions)
    classes = ['Acral Lentiginous Melanoma', 'Beaus Line', 'Blue Finger', 'Clubbing',
              'Healthy Nail', 'Koilonychia', 'Muehrckes Lines', 'Onychogryphosis',
              'Pitting', 'Terry-s Nail']
    print(f"Predicted class: {classes[predicted_class]}")
  
    accuracy = predictions[0][predicted_class] * 100.0
    print(f"Prediction accuracy: {accuracy:.2f}%")
  
  else:
    print('It is not nails')

In [6]:
inference_model('./Inference_model/car.jpeg')

1/1 [==============================] - 3s 3s/step
It is not nails


In [7]:
inference_model('./Inference_model/healthy.jpeg')

1/1 [==============================] - 1s 594ms/step
Predicted class: Healthy Nail
Prediction accuracy: 74.25%


In [8]:
interpreter2 = tf.lite.Interpreter(model_path='./saved_models/my_model.tflite')
interpreter2.allocate_tensors()

In [9]:
interpreter1 = tf.lite.Interpreter(model_path='./saved_models/keras_model.tflite')
interpreter1.allocate_tensors()

In [10]:
def inference_model_tflite(path):
    img = path
    img = image.load_img(img, target_size=(224, 224))  # Adjust target size as per your model's input shape
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    input_details1 = interpreter1.get_input_details()
    output_details1 = interpreter1.get_output_details()
    interpreter1.set_tensor(input_details1[0]['index'], img_array)
    interpreter1.invoke()
    output_data = interpreter1.get_tensor(output_details1[0]['index'])
    classes = ['nails', 'not nails']
    predicted_class = np.argmax(output_data)
    print(f"Predicted class: {classes[predicted_class]}")
    if classes[predicted_class] == 'nails':
        input_details2 = interpreter1.get_input_details()
        output_details2 = interpreter1.get_output_details()
        interpreter2.set_tensor(input_details2[0]['index'], img_array)
        interpreter2.invoke()
        output_data = interpreter2.get_tensor(output_details2[0]['index'])
        classes = ['Acral Lentiginous Melanoma', 'Beaus Line', 'Blue Finger', 'Clubbing',
              'Healthy Nail', 'Koilonychia', 'Muehrckes Lines', 'Onychogryphosis',
              'Pitting', 'Terry-s Nail']
        predicted_class = np.argmax(output_data)
        print(f"Predicted class: {classes[predicted_class]}")
        
        accuracy = output_data[0][predicted_class] * 100.0
        print(f"Prediction accuracy: {accuracy:.2f}%")
    else:
       print('It is not nails')

In [11]:
inference_model_tflite('./Inference_model/healthy.jpeg')

Predicted class: nails
Predicted class: Healthy Nail
Prediction accuracy: 73.54%


In [12]:
inference_model('./Inference_model/car.jpeg')

1/1 [==============================] - 0s 23ms/step
It is not nails


In [13]:
inference_model('./Inference_model/laut.jpeg')

1/1 [==============================] - 0s 23ms/step
It is not nails
